In [1]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy import signal
import os
import pandas as pd
from scipy.signal import stft
from math import prod
# from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from math import e
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder
from urllib.request import urlopen, urlretrieve
from itertools import combinations

# import matplotlib.pyplot as plt
%matplotlib inline

2022-06-23 13:38:15.228973: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-06-23 13:38:15.232305: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-23 13:38:15.232328: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def butter_bandpass(data, lowcut, highcut, fs=200, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)


def butter_lowpass(data, lowcut, fs=200, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    b, a = signal.butter(order, low, btype='lowpass')
    return signal.filtfilt(b, a, data)


def butter_highpass(data, highcut, fs=200, order=4):
    nyq = fs * 0.5
    high = highcut / nyq
    b, a = signal.butter(order, high, btype='highpass')
    return signal.filtfilt(b, a, data)


def butter_notch(data, cutoff, var=1, fs=200, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

In [3]:
def filtros(data):
    data_filtered = butter_notch(data, 60)
    data_filtered = butter_highpass(data_filtered, 5)
    data_filtered = butter_lowpass(data_filtered, 50)
    return data_filtered

In [4]:
dir = './datasets/topicos_cc'
arquivos = os.listdir(dir)
arq_numpy = [f for f in arquivos if f.endswith(".npy") and f.startswith('p2')]
participantes = {}
vetor = []
for i in arq_numpy:
    nome = i.split('_')
    trial = np.load(dir+'/'+i)
    for m in range(0,8):
        if participantes.get(f'trial_{nome[1]}',0) == 0:
            participantes[f'trial_{nome[1]}'] = []
        dados = trial[m, :, :].swapaxes(0,1)
        participantes[f'trial_{nome[1]}'].append(filtros(dados))
        

arr = np.vstack((np.array(participantes['trial_1']), np.array(participantes['trial_2']), np.array(participantes['trial_3'])))


print(arr.shape)

(24, 4, 1600)


In [5]:
def create_chunks(data):    
    step = 11.8
    segment = 128
    # data = arr
    # .get_data()
    print('', data.shape)

    n_win = int((data.shape[-1] - segment) / step) + 1
    ids = np.arange(n_win) * int(step)

    # Janelas do dado no dominio do tempo
    chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

    # Janelas do dado no domínio da frequência
    _, _, chunks_freq = stft(data, fs=200, nperseg=128, noverlap=115)
    chunks_freq = np.swapaxes(chunks_freq, 2, 3)

    print('Formato (shape) dos dados depois da divisão de janelas')
    print(f'Dominio do tempo: {chunks_time.shape} - (classes, ensaios, canais, janelas, linhas)')
    print(f'Dominio da frequência:  {chunks_freq.shape} - (classes, ensaios, canais, janelas, linhas)')
    return chunks_time, chunks_freq

In [6]:
# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2


# funções de extração de características

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape[:-1]) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape[:-1])))

def wamp(x):
    limiar = np.abs(np.diff(x))
    return np.sum(limiar > 0.0001, axis=-1)

def wl(x):
    return np.sum(np.abs(np.diff(x)), axis=-1)


def getzc(data, th):
    t = len(data)
    soma = 0
    for i in range(t-1):
        res = (data[i]*data[i+1])
        res2 = np.abs(data[i]-data[i+1])
        if (res<0 and res2 > th):
            soma +=1
    return soma

def zc(data):
    f=[]
    x,y,z = data.shape[:3]
    for i in range(x):
        l = []
        for j in range(y):
            li = []
            for k in range(z):
                li.append(getzc(data[i][j][k], 0.0001))
            l.append(li.copy())
        f.append(l.copy())

    return np.array(f)

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2


def fmn(w):
    sample_rate = 200
    f = (w * sample_rate)/(2*len(w))
    return np.sum(np.abs(f*PSD(w)), axis=-1)/np.sum(PSD(w), axis=-1)

def mmnf(w):
    sample_rate = 200
    f = (w * sample_rate)/(2*len(w))
    return np.sum(np.abs(f*np.abs(w)), axis=-1)/np.sum(np.abs(w), axis=-1)


def logDec(data):
    N = np.prod(data.shape)
    return e ** (np.sum(np.log10(np.abs(data)), axis=-1))/N


def iemg(x):
    return np.sum(np.abs(x), axis=-1)

def dasdv(x):
    return np.sqrt(np.sum(np.diff(x)**2, axis=-1)/(np.prod(x.shape[:-1]) - 1))

def tm(x,n):
    return np.abs(np.sum(x**n , axis=-1)/np.prod(x.shape[:-1]))

In [7]:
def gen_features(chunks_time, chunks_freq, data):
    final_data = list()
    final_data.append(var(chunks_time))
    final_data.append(rms(chunks_time))
    final_data.append(fmd(chunks_freq))
    final_data.append(mmdf(chunks_freq))
    final_data.append(logDec(chunks_time))
    final_data.append(wamp(chunks_time))
    final_data.append(wl(chunks_time))
    final_data.append(zc(chunks_time))
    final_data.append(fmn(chunks_freq))
    final_data.append(mmnf(chunks_freq))
    final_data.append(iemg(chunks_time))
    final_data.append(dasdv(chunks_time))
    
    for n in range(3,6):
        final_data.append(tm(chunks_time, n))
    
    f, Pxx_den = signal.welch(data, fs=200, nperseg=248, noverlap=223)
    final_data.append(Pxx_den)
    
    final = np.array(final_data)
    print(final.shape)
    return final


In [8]:
trial = []
trial.append(arr[:8])
trial.append(arr[8:16])
trial.append(arr[16:24])

new_array = []

for i in trial:
    time, freq = create_chunks(i)
    new_array.append(gen_features(time, freq, i))

# new_array = np.vstack((np.array(new_array[0]), np.array(new_array[1]), np.array(new_array[2])))
new_array=np.array(new_array)
new_array=new_array.reshape(new_array.shape[1], new_array.shape[2]*new_array.shape[0],new_array.shape[3], new_array.shape[4])
new_array.shape

 (8, 4, 1600)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (8, 4, 125, 128) - (classes, ensaios, canais, janelas, linhas)
Dominio da frequência:  (8, 4, 125, 65) - (classes, ensaios, canais, janelas, linhas)
(16, 8, 4, 125)
 (8, 4, 1600)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (8, 4, 125, 128) - (classes, ensaios, canais, janelas, linhas)
Dominio da frequência:  (8, 4, 125, 65) - (classes, ensaios, canais, janelas, linhas)
(16, 8, 4, 125)
 (8, 4, 1600)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (8, 4, 125, 128) - (classes, ensaios, canais, janelas, linhas)
Dominio da frequência:  (8, 4, 125, 65) - (classes, ensaios, canais, janelas, linhas)
(16, 8, 4, 125)


(16, 24, 4, 125)

In [9]:
X = new_array.transpose(1,3,2,0)
X = X.reshape(X.shape[0]*X.shape[1], X.shape[2]*X.shape[3])
X.shape

(3000, 64)

In [10]:
y = [[i] * int(X.shape[0] / 8) for i in range(8)]
y = np.array(y).flatten()
y.shape

(3000,)

In [11]:
trial = []
trial.append((X[:1000], y[:1000]))
trial.append((X[1000:2000], y[1000:2000]))
trial.append((X[2000:3000], y[2000:3000]))

In [12]:
def plot_history(h):
    loss_list = [s for s in h.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in h.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in h.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in h.history.keys() if 'acc' in s and 'val' in s]
    if len(loss_list) == 0:
        print('Custo não está presente no histórico')
        return
    epochs = range(1, len(history.history[loss_list[0]]) + 1)
    # Custo
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, h.history[l], 'b',
                 label='Custo [treinamento] (' + str(str(format(
                    h.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, h.history[l], 'g',
                 label='Custo [validação] (' + str(str(format(
                    h.history[l][-1],'.5f'))+')'))
    plt.title('Custo')
    plt.xlabel('Épocas')
    plt.ylabel('Custo')
    plt.legend()
    # Acurácia
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, h.history[l], 'b',
                 label='Acurácia [treinamento] (' + str(format(
                    h.history[l][-1],'.5f'))+')')
    for l in val_acc_list:
        plt.plot(epochs, h.history[l], 'g',
                 label='Acurácia [validação] (' + str(format(
                    h.history[l][-1],'.5f'))+')')
    plt.title('Acurácia')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.show()

In [13]:
def cross_val(model):
    j=0
    scores = []
    validations=[]
    for i in combinations(trial, 2):
        X_treino = np.vstack((i[0][0],i[1][0]))
        X_teste = trial[j][0]

        y_treino = np.vstack((i[0][1],i[1][1]))
        y_treino= y_treino.reshape(y_treino.shape[0]*y_treino.shape[1])
        y_teste = trial[j][1]


        j+=1

        # X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.3, shuffle=True, stratify=y_treino)
        X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.3, stratify=y_treino)


        scaler = StandardScaler()
        scaler.fit(X_treino)

        X_treino = scaler.transform(X_treino)
        X_teste = scaler.transform(X_teste)
        X_val = scaler.transform(X_val)

        history = model.fit(X_treino, y_treino, epochs=100, batch_size=30, validation_data=(X_val, y_val))
        # plot_history(history)
        predict_x=model.predict(X_teste) 
        score=np.argmax(predict_x,axis=1)

        # print('--',score)

        scores.append(score)
        validations.append(y_teste)


    # y_true =  np.vstack((trial[0][1],trial[1][1], trial[2][1]))
    y_true =  np.vstack((validations[0],validations[1], validations[2]))
    y_true= y_true.reshape(y_true.shape[0]*y_true.shape[1])
    sc = np.vstack((scores[0], scores[1], scores[2]))
    sc= sc.reshape(sc.shape[0]*sc.shape[1])

    print('\n\nAcurácia: %0.2f%%' % (accuracy_score(y_true, sc) * 100))
    # print('Matriz de confusão:')
    # print(confusion_matrix(y_true, sc))
    # print()
    # print(classification_report(y_true, sc, digits=5))

In [14]:

def execute_mlp(b):
    # definição de uma fração do regularizador
    l = 1E-5

    # desenvolvimento do modelo Keras para uma MLP
    model = Sequential()
    model.add(Dense(20, activation='relu', input_dim=64,
                    kernel_regularizer=regularizers.l2(l)))
    # Aplicação de um dropout (caso necessário)
    # model.add(Dropout(d))
    model.add(Dense(10, activation='relu',
                    kernel_regularizer=regularizers.l2(l)))
    # Aplicação de um dropout (caso necessário)
    # model.add(Dropout(d))
    model.add(Dense(8, activation='softmax'))

    # Aplicação de um modelo de descida de gradiente utilizando o Stocastic Gradient Descendent (SGD)
    # sgd = SGD(learning_rate=lr, momentum=0.0)
    # Função de otimização da rede: ADAM
    adam = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)
    # adam = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999)
    # Função de custo baseada em dados originalmente categóricos


    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam,
                metrics=['accuracy'])


    
    cross_val(model)

SVM: kernel, C e gamma;
MLP: valor de dropout, batch size e quantidade de épocas.

In [15]:
learning_rates = [1E-0, 1E-1, 1E-2, 1E-3, 1E-4, 1E-5]
# melhor valor lrate Adam 1E-2 6470, sgd 1E-1 foi pior 6383
learning_rates

[1.0, 0.1, 0.01, 0.001, 0.0001, 1e-05]

In [16]:
penaltis =  [1E-0, 1E-1, 1E-2, 1E-3, 1E-4, 1E-5]
# melhor l2 1E-5 66.47;

In [17]:
drop =[0.1,0.2,0.3,0.4,0.5]
# melhor sem dropout

In [18]:
[1E-0, 10, 25, 50, 100, 300, 500, 1000]

[1.0, 10, 25, 50, 100, 300, 500, 1000]

In [19]:
batchs = list(np.linspace(10, 100, 10))
# batchs = [10, 25, 50, 100, 300, 500, 1000]
# melhor batch 30
batchs=[int(i) for i in batchs]
batchs

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [20]:
epoca = [1, 10, 25, 50, 100, 300, 500, 1000]
# melhor epochs 1000

In [21]:
execute_mlp(i)

Epoch 1/100


2022-06-23 13:38:18.815392: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-23 13:38:18.815423: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-23 13:38:18.815443: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (luisotavio-Aspire5): /proc/driver/nvidia/version does not exist
2022-06-23 13:38:18.815674: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


47/47 [==============================] - 0s 3ms/step - loss: 1.2345 - accuracy: 0.5357 - val_loss: 6.4671 - val_accuracy: 0.7133
Epoch 2/100
47/47 [==============================] - 0s 1ms/step - loss: 0.5722 - accuracy: 0.7621 - val_loss: 0.5293 - val_accuracy: 0.8150
Epoch 3/100
47/47 [==============================] - 0s 1ms/step - loss: 0.3924 - accuracy: 0.8664 - val_loss: 0.3804 - val_accuracy: 0.8633
Epoch 4/100
47/47 [==============================] - 0s 1ms/step - loss: 0.2973 - accuracy: 0.8929 - val_loss: 0.3089 - val_accuracy: 0.8800
Epoch 5/100
47/47 [==============================] - 0s 1ms/step - loss: 0.2514 - accuracy: 0.9079 - val_loss: 0.2572 - val_accuracy: 0.9100
Epoch 6/100
47/47 [==============================] - 0s 1ms/step - loss: 0.2144 - accuracy: 0.9264 - val_loss: 0.2595 - val_accuracy: 0.8800
Epoch 7/100
47/47 [==============================] - 0s 1ms/step - loss: 0.2163 - accuracy: 0.9207 - val_loss: 0.2916 - val_accuracy: 0.8817
Epoch 8/100
47/47 [======

In [22]:
# for i in batchs:
#     execute_mlp(i)

Epoch 1/100


2022-06-23 13:28:57.118830: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-23 13:28:57.118873: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-23 13:28:57.118903: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (luisotavio-Aspire5): /proc/driver/nvidia/version does not exist
2022-06-23 13:28:57.119168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


140/140 [==============================] - 1s 2ms/step - loss: 0.9628 - accuracy: 0.6114 - val_loss: 120.3164 - val_accuracy: 0.7483
Epoch 2/100
140/140 [==============================] - 0s 1ms/step - loss: 0.4979 - accuracy: 0.7921 - val_loss: 0.4641 - val_accuracy: 0.8467
Epoch 3/100
140/140 [==============================] - 0s 1ms/step - loss: 0.3206 - accuracy: 0.8736 - val_loss: 0.3194 - val_accuracy: 0.8817
Epoch 4/100
140/140 [==============================] - 0s 1ms/step - loss: 0.2574 - accuracy: 0.8936 - val_loss: 0.7099 - val_accuracy: 0.8917
Epoch 5/100
140/140 [==============================] - 0s 1ms/step - loss: 0.2045 - accuracy: 0.9207 - val_loss: 1.0864 - val_accuracy: 0.9167
Epoch 6/100
140/140 [==============================] - 0s 1ms/step - loss: 0.2226 - accuracy: 0.9114 - val_loss: 1.3295 - val_accuracy: 0.8783
Epoch 7/100
140/140 [==============================] - 0s 1ms/step - loss: 0.1687 - accuracy: 0.9371 - val_loss: 0.8765 - val_accuracy: 0.9217
Epoch 8/1

In [14]:
def cross_val2(model):
    j=0
    scores = []
    validations = []
    for i in combinations(trial, 2):
        X_treino = np.vstack((i[0][0],i[1][0]))
        X_teste = trial[j][0]

        y_treino = np.vstack((i[0][1],i[1][1]))
        y_treino= y_treino.reshape(y_treino.shape[0]*y_treino.shape[1])
        y_teste = trial[j][1]


        j+=1

        X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.3, stratify=y_treino)

        scaler = StandardScaler()
        scaler.fit(X_treino)

        X_treino = scaler.transform(X_treino)
        X_teste = scaler.transform(X_teste)
        X_val = scaler.transform(X_val)

        history = model.fit(X_treino, y_treino)
        # plot_history(history)
        predict_x=model.predict(X_val) 
        # print(predict_x)
        # print(predict_x.shape)
        # score=np.argmax(predict_x)
        # score=np.argmax(predict_x,axis=1)
        # scores.append(score)
        scores.append(predict_x)
        validations.append(y_val)

    y_true =  np.vstack((validations[0],validations[1], validations[2]))
    y_true= y_true.reshape(y_true.shape[0]*y_true.shape[1])
    sc = np.vstack((scores[0], scores[1], scores[2]))
    sc= sc.reshape(sc.shape[0]*sc.shape[1])

    print('\n\nAcurácia: %0.2f%%' % (accuracy_score(y_true, sc) * 100))
    print('Matriz de confusão:')
    print(confusion_matrix(y_true, sc))
    print()
    print(classification_report(y_true, sc, digits=5))

In [15]:

# modelo de classificador com os parâmetros padrões
clf = SVC(gamma='scale')

# criando o modelo de classificação com os dados de treino
cross_val2(clf)



Acurácia: 65.17%
Matriz de confusão:
[[153   8  24  19   2   6  11   2]
 [  4 119   4  16  28   0  38  16]
 [ 13  21 144   4  20   8   0  15]
 [  8  11  12 134   7  10  43   0]
 [  8  21   8  21 136   4  13  15]
 [ 11   3  13  21   6 128  29  13]
 [  0   1   0   9   2   5 207   1]
 [ 10   8   6  10  20  14   5 152]]

              precision    recall  f1-score   support

           0    0.73913   0.68000   0.70833       225
           1    0.61979   0.52889   0.57074       225
           2    0.68246   0.64000   0.66055       225
           3    0.57265   0.59556   0.58388       225
           4    0.61538   0.60177   0.60850       226
           5    0.73143   0.57143   0.64160       224
           6    0.59827   0.92000   0.72504       225
           7    0.71028   0.67556   0.69248       225

    accuracy                        0.65167      1800
   macro avg    0.65867   0.65165   0.64889      1800
weighted avg    0.65861   0.65167   0.64887      1800

